In [13]:
import sys
print(sys.executable)
print(sys.version)

/Library/Developer/CommandLineTools/usr/bin/python3
3.9.6 (default, Oct 17 2025, 17:15:53) 
[Clang 17.0.0 (clang-1700.4.4.1)]


In [14]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 275.9 kB/s  0:00:061.5 kB/s eta 0:00:01:05
  Attempting uninstall: pip
    Found existing installation: pip 26.0
    Uninstalling pip-26.0:
      Successfully uninstalled pip-26.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install tensorflow-macos==2.15.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install tensorflow-metal==1.1.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### 1. Importing Dataset and Data Augmentation

In [17]:
## 4 classes
import warnings
from tensorflow.keras.preprocessing.image import ImageDataGenerator

warnings.filterwarnings("ignore")

train_dir= "./alzheimer dataset/train"
test_dir ="./alzheimer dataset/test"

## object for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale= 1/.255, ##normalize pixel values
    shear_range = 0.1 , ## slant range, titling medical images might harm its quality
    zoom_range = 0.2 , ## zoom range for scale 
    width_shift_range = 0.1 , ##moving image left to right
    height_shift_range = 0.1, ##moving image top to bottom
    rotation_range = 0 , ## this might harm medical image classification
    horizontal_flip =False,
    fill_mode = 'nearest'
)

test_datagen = ImageDataGenerator(rescale =1/.255)

##load the images ,labels , ensuring grayscale (1 color channel)
train_generator = train_datagen.flow_from_directory(
    train_dir ,
    target_size =(128, 128), ##resize the images 
    color_mode ='grayscale',
    class_mode='categorical',
    
) # default batch size =32

test_generator =test_datagen.flow_from_directory(
    test_dir,
    target_size=(128,128),
    color_mode ='grayscale',
    class_mode='categorical'
)


Found 4649 images belonging to 4 classes.
Found 1813 images belonging to 4 classes.


In [18]:
4649/32 #steps per batch

145.28125

### 2. Model Building -CNN 

In [19]:
# from keras.models import Sequential
# from keras.layers import Conv2D
# from keras.layers import Flatten
# from keras.layers import MaxPooling2D
# from keras.layers import Dense

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , MaxPooling2D ,Flatten ,Dense ,Dropout
import numpy as np
from tensorflow.keras.optimizers import Adam

In [21]:
##define the model
CNN1 = Sequential()
CNN1.add(Conv2D(256,(3,3),activation ='relu',padding='same',input_shape=(128,128,1)))
CNN1.add(MaxPooling2D(pool_size=(2,2))) 

CNN1.add(Conv2D(64,(3,3),activation ='relu',padding ='same'))
CNN1.add(MaxPooling2D(pool_size=(2,2)))

CNN1.add(Conv2D(256,(3,3),activation ='relu',padding ='same'))
CNN1.add(MaxPooling2D(pool_size=(2,2)))

CNN1.add(Flatten())
##after feature extraction using convolution layers and flattening the result for FC layers
CNN1.add(Dense(128,activation='relu'))
CNN1.add(Dropout(0.2))
CNN1.add(Dense(64,activation ='relu'))

##output layer
CNN1.add(Dense(4,activation ='softmax'))

##compile the model
CNN1.compile(optimizer=Adam(learning_rate=0.0001, clipnorm=1.0) , loss='categorical_crossentropy', metrics=['accuracy'])

##Model summary
CNN1.summary()


# Reduce batch size
train_generator.batch_size = 16

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 128, 128, 256)     2560      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 64, 64, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 64, 64, 64)        147520    
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 32, 32, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 32, 32, 256)       147712    
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 16, 16, 256)      

In [22]:
(3*3+1)*256

2560

In [23]:
CNN1.fit(train_generator , epochs=50)

Epoch 1/50


2026-02-08 20:23:40.823947: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


291/291 [==============================] - 18s 54ms/step - loss: 67.2733 - accuracy: 0.4293
Epoch 2/50
291/291 [==============================] - 13s 46ms/step - loss: 5801.7212 - accuracy: 0.4173
Epoch 3/50
291/291 [==============================] - 12s 42ms/step - loss: 10178.8896 - accuracy: 0.4543
Epoch 4/50
291/291 [==============================] - 12s 43ms/step - loss: 12277.4775 - accuracy: 0.4438
Epoch 5/50
291/291 [==============================] - 12s 42ms/step - loss: 8836.3809 - accuracy: 0.4524
Epoch 6/50
291/291 [==============================] - 12s 42ms/step - loss: 5173.5029 - accuracy: 0.4560
Epoch 7/50
291/291 [==============================] - 12s 42ms/step - loss: 3594.1160 - accuracy: 0.4431
Epoch 8/50
291/291 [==============================] - 12s 42ms/step - loss: 3303.6985 - accuracy: 0.4521
Epoch 9/50
291/291 [==============================] - 12s 42ms/step - loss: 5379.9673 - accuracy: 0.4438
Epoch 10/50
291/291 [==============================] - 12s 41ms/st

In [24]:
loss, accuracy1 = CNN1.evaluate(test_generator)

print(f"Test accuracy of the model : {accuracy1}")
##with padding for 50 epochs -53 %
##without padding for 20 epochs -45 %
##without padding for 50 epochs - 53 %

57/57 [==============================] - 5s 43ms/step - loss: 48429.9375 - accuracy: 0.5334
Test accuracy of the model : 0.5333700776100159


### 3. With Batch Normalization

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , MaxPooling2D ,Flatten ,Dense ,Dropout,BatchNormalization

##define the model
CNN2 = Sequential()
CNN2.add(Conv2D(256,(3,3),activation ='relu',padding='valid',input_shape=(128,128,1)))
CNN2.add(BatchNormalization())
CNN2.add(MaxPooling2D(pool_size=(2,2))) 

CNN2.add(Conv2D(64,(3,3),activation ='relu',padding ='valid'))
CNN2.add(BatchNormalization())
CNN2.add(MaxPooling2D(pool_size=(2,2)))

CNN2.add(Conv2D(256,(3,3),activation ='relu',padding ='valid'))
CNN2.add(BatchNormalization())
CNN2.add(MaxPooling2D(pool_size=(2,2)))

CNN2.add(Flatten())
##after feature extraction using convolution layers and flattening the result for FC layers
CNN2.add(Dense(128,activation='relu'))
CNN2.add(BatchNormalization())
CNN2.add(Dropout(0.2))

CNN2.add(Dense(64,activation ='relu'))
CNN2.add(BatchNormalization())
##output layer
CNN2.add(Dense(4,activation ='softmax'))

##compile the model
CNN2.compile(optimizer='adam' , loss='categorical_crossentropy', metrics=['accuracy'])

##Model summary
CNN2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 126, 126, 256)     2560      
                                                                 
 batch_normalization (Batch  (None, 126, 126, 256)     1024      
 Normalization)                                                  
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 63, 63, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 61, 61, 64)        147520    
                                                                 
 batch_normalization_1 (Bat  (None, 61, 61, 64)        256       
 chNormalization)                                                
                                                      

In [26]:
CNN2.fit(train_generator , epochs=50)

Epoch 1/50
291/291 [==============================] - 16s 51ms/step - loss: 1.0685 - accuracy: 0.5700
Epoch 2/50
291/291 [==============================] - 15s 51ms/step - loss: 0.8548 - accuracy: 0.6283
Epoch 3/50
291/291 [==============================] - 15s 52ms/step - loss: 0.8177 - accuracy: 0.6335
Epoch 4/50
291/291 [==============================] - 15s 51ms/step - loss: 0.8090 - accuracy: 0.6384
Epoch 5/50
291/291 [==============================] - 15s 52ms/step - loss: 0.8040 - accuracy: 0.6404
Epoch 6/50
291/291 [==============================] - 15s 51ms/step - loss: 0.7913 - accuracy: 0.6401
Epoch 7/50
291/291 [==============================] - 15s 51ms/step - loss: 0.7846 - accuracy: 0.6451
Epoch 8/50
291/291 [==============================] - 15s 52ms/step - loss: 0.7779 - accuracy: 0.6490
Epoch 9/50
291/291 [==============================] - 15s 52ms/step - loss: 0.7617 - accuracy: 0.6556
Epoch 10/50
291/291 [==============================] - 17s 58ms/step - loss: 0.768

In [27]:
loss, accuracy2 = CNN2.evaluate(test_generator)

print(f"Test accuracy of the model : {accuracy2}")

57/57 [==============================] - 2s 26ms/step - loss: 1.6119 - accuracy: 0.4495
Test accuracy of the model : 0.4495311677455902


### 4. With L1 Regularization

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , MaxPooling2D ,Flatten ,Dense ,Dropout,BatchNormalization
from tensorflow.keras import regularizers

##define the model
CNN3 = Sequential()
CNN3.add(Conv2D(256,(3,3),activation ='relu',padding='valid',input_shape=(128,128,1),
               kernel_regularizer = regularizers.l1(0.001)))
CNN3.add(BatchNormalization())
CNN3.add(MaxPooling2D(pool_size=(2,2))) 

CNN3.add(Conv2D(64,(3,3),activation ='relu',padding ='valid'))
CNN3.add(BatchNormalization())
CNN3.add(MaxPooling2D(pool_size=(2,2)))

CNN3.add(Conv2D(256,(3,3),activation ='relu',padding ='valid'))
CNN3.add(BatchNormalization())
CNN3.add(MaxPooling2D(pool_size=(2,2)))

CNN3.add(Flatten())
##after feature extraction using convolution layers and flattening the result for FC layers
CNN3.add(Dense(128,activation='relu',kernel_regularizer = regularizers.l1(0.001)))
CNN3.add(BatchNormalization())
CNN3.add(Dropout(0.2))

CNN3.add(Dense(64,activation ='relu',kernel_regularizer = regularizers.l1(0.001)))
CNN3.add(BatchNormalization())
##output layer
CNN3.add(Dense(4,activation ='softmax'))

##compile the model
CNN3.compile(optimizer='adam' , loss='categorical_crossentropy', metrics=['accuracy'])

##Model summary
CNN3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 126, 126, 256)     2560      
                                                                 
 batch_normalization_5 (Bat  (None, 126, 126, 256)     1024      
 chNormalization)                                                
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 63, 63, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 61, 61, 64)        147520    
                                                                 
 batch_normalization_6 (Bat  (None, 61, 61, 64)        256       
 chNormalization)                                                
                                                      

In [30]:
CNN3.fit(train_generator , epochs=50)

Epoch 1/50
291/291 [==============================] - 17s 55ms/step - loss: 7.9257 - accuracy: 0.5143
Epoch 2/50
291/291 [==============================] - 16s 55ms/step - loss: 5.3656 - accuracy: 0.5855
Epoch 3/50
291/291 [==============================] - 16s 56ms/step - loss: 5.0256 - accuracy: 0.5881
Epoch 4/50
291/291 [==============================] - 17s 58ms/step - loss: 4.6232 - accuracy: 0.5935
Epoch 5/50
291/291 [==============================] - 16s 54ms/step - loss: 4.6791 - accuracy: 0.5904
Epoch 6/50
291/291 [==============================] - 16s 54ms/step - loss: 4.0456 - accuracy: 0.5952
Epoch 7/50
291/291 [==============================] - 16s 56ms/step - loss: 3.8883 - accuracy: 0.5975
Epoch 8/50
291/291 [==============================] - 16s 55ms/step - loss: 3.8171 - accuracy: 0.5993
Epoch 9/50
291/291 [==============================] - 16s 54ms/step - loss: 3.4831 - accuracy: 0.6051
Epoch 10/50
291/291 [==============================] - 18s 61ms/step - loss: 3.277

In [31]:
loss, accuracy3 = CNN3.evaluate(test_generator)

print(f"Test accuracy of the model : {accuracy3}")

57/57 [==============================] - 2s 26ms/step - loss: 2.9053 - accuracy: 0.2438
Test accuracy of the model : 0.24379481375217438


### 5.with batch normalization ,l2 regularization ,dropout

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , MaxPooling2D ,Flatten ,Dense ,Dropout,
BatchNormalization,Activation
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

CNN4 =Sequential()
##block 1
CNN4.add(Conv2D(256,(3,3) , input_shape = (12,128,1),
               kernel_regularizer =regularizers.l2(0.001)))

CNN4.add(BatchNormalization())
CNN4.add(Activation('relu'))
CNN4.add(MaxPooling2D(pool_size=(2,2)))
CNN4.add(Dropout(0.25))

##block2 
CNN4.add(Conv2D(64,(3,3),kernel_regularizer =regularizers.l2(0.001)))

CNN4.add(BatchNormalization())
CNN4.add(Activation('relu'))
CNN4.add(MaxPooling2D(pool_size=(2,2)))
CNN4.add(Dropout(0.25)) ##small dropout
##classifier

CNN4.add(Flatten())
CNN4.add(Dense(128 , kernel_regularizer = regularizers.l2(0.001)))
CNN4.add(BatchNormalization())
CNN4.add(Activation('relu'))
CNN4.add(Dropout(0.5))


CNN4.add(Dense(64 , kernel_regularizer = regularizers.l2(0.001)))
CNN4.add(BatchNormalization())
CNN4.add(Activation('relu'))
CNN4.add(Dropout(0.5)) ## large dropout

CNN4.compile(optimizer =Adam(learning_rate =0.0004),
            loss ='categorical_crossentropy',
            metrics=['accuracy'])

CNN4.summary()

SyntaxError: trailing comma not allowed without surrounding parentheses (1397012191.py, line 2)

In [ ]:
CNN4.fit(train_generator , epochs=50)

In [ ]:
loss, accuracy4 = CNN4.evaluate(test_generator)

print(f"Test accuracy of the model : {accuracy4}")